# 10.1 自然语言理解

第9章中描述的基于特征的文法形式可以很容易地从英语翻译到SQL。文法sql0.fcfg 说明如何将句子意思表示与句子分析串联组装。
每个短语结构规则为特征SEM 构建值作补充。你可以看到这些补充非常简单；在每一种情况下，我们对分割的子成分用字符串连接
操作+来组成父成分的值。

In [2]:
import nltk  
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')  

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


这使我们能够分析SQL 查询

In [3]:
from nltk import load_parser
cp = load_parser('grammars/book_grammars/sql0.fcfg')
query = 'What cities are located in China'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
q = ' '.join(answer)
print(q)

SELECT City FROM city_table WHERE Country="china"


最后，我们在数据库city.db 上执行查询，检索出一些结果

In [4]:
from nltk.sem import chat80
rows = chat80.sql_query('corpora/city_database/city.db', q)
for r in rows: print(r[0], end=" ")

canton chungking dairen harbin kowloon mukden peking shanghai sian tientsin 

我们将重点探索NLTK 中的逻辑表示方式，所以将使用下列ASCII 版本的运算符：

In [5]:
nltk.boolean_ops()

negation       	-
conjunction    	&
disjunction    	|
implication    	->
equivalence    	<->


NLTK 的LogicParser()将逻辑表达式分析成表达式的各种子类：

In [6]:
read_expr = nltk.sem.Expression.fromstring
read_expr('-(P & Q)')

<NegatedExpression -(P & Q)>

In [7]:
read_expr('P & Q')

<AndExpression (P & Q)>

In [8]:
read_expr('P | (R -> Q)')

<OrExpression (P | (R -> Q))>

In [9]:
read_expr('P <-> -- P')

<IffExpression (P <-> --P)>

NLTK 中的inference 模块通过一个第三方定理证明器Prover9 的接口，可以进行
逻辑证明。推理机制的输入首先必须用LogicParser()分析成逻辑表达式。

In [11]:
lp = nltk.sem.Expression.fromstring
SnF = read_expr('SnF')
NotFnS = read_expr('-FnS')
R = read_expr('SnF -> -FnS')
prover = nltk.Prover9()
prover.prove(NotFnS, [SnF, R])

LookupError: 

===========================================================================
NLTK was unable to find the prover9 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on prover9, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

一个命题逻辑的模型需要为每个可能的公式分配值True 或False。我们一
步步的来做这个：首先，为每个命题符号分配一个值，然后确定布尔运算符的含义（即表1
0-2）和运用它们到这些公式的组件的值，来计算复杂的公式的值。估值是从逻辑的基本符
号映射到它们的值

In [12]:
 val = nltk.Valuation([('P', True), ('Q', True), ('R', False)])

我们使用一个配对的链表初始化一个估值，每个配对由一个语义符号和一个语义值组
成。所产生的对象基本上只是一个字典，映射逻辑符号（作为字符串处理）为适当的值。

In [13]:
val['P']

True

In [15]:
dom = set()
g = nltk.Assignment(dom)

让我们用val 初始化模型m：

In [16]:
m = nltk.Model(dom, val)

每一个模型都有一个evaluate()方法，可以确定逻辑表达式，如命题逻辑的公式，的
语义值；当然，这些值取决于最初我们分配给命题符号如P、Q 和R 的真值。

In [17]:
 print(m.evaluate('(P & Q)', g))

True


In [18]:
print(m.evaluate('-(P & Q)', g))

False


In [19]:
print(m.evaluate('(P & R)', g))

False


In [20]:
print(m.evaluate('(P | R)', g))

True


逻辑表达式可以通过类型检查进行处理

In [21]:
read_expr = nltk.sem.Expression.fromstring
expr = read_expr('walk(angus)', type_check=True)
expr.argument

<ConstantExpression angus>

In [22]:
expr.argument.type

e

In [23]:
expr.function

<ConstantExpression walk>

In [24]:
expr.function.type

<e,?>

In [ ]:
我们需要指定一个信号，作为一个字典来实施，明确的与非逻辑常量类型关联：

In [25]:
sig = {'walk': '<e, t>'} ##制定信号，将其作为字典实现与非逻辑常量类型之间的关联
expr = read_expr('walk(angus)', signature=sig)
expr.function.type

e

NLTK 中的LogicParser 的parse()方法返回Expression 类的
对象。这个类的每个实例expr 都有free()方法，返回一个在expr 中自由的变量的集合

In [26]:
read_expr = nltk.sem.Expression.fromstring

NLTK 中的LogicParser 的parse()方法返回Expression 类的
对象。这个类的每个实例expr 都有free()方法，返回一个在expr 中自由的变量的集合。

In [27]:
read_expr('dog(cyril)').free()  #特指Cyril

set()

In [28]:
read_expr('dog(x)').free()   #自由变量x

{Variable('x')}

In [29]:
read_expr('own(angus, cyril)').free()

set()

In [30]:
read_expr('exists x.dog(x)').free()  #封闭的

set()

In [31]:
read_expr('((some x. walk(x)) -> sing(x))').free()

{Variable('x')}

In [32]:
read_expr('exists x.own(y, x)').free()   #自由变量y

{Variable('y')}

一阶定理证明

In [33]:
NotFnS = read_expr('-north_of(f, s)') 
SnF = read_expr('north_of(s, f)')    
R = read_expr('all x. all y. (north_of(x, y) -> -north_of(y, x))')  
prover = nltk.Prover9()   
prover.prove(NotFnS, [SnF, R]) 

LookupError: 

===========================================================================
NLTK was unable to find the prover9 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on prover9, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [34]:
FnS = read_expr('north_of(f, s)')
prover.prove(FnS, [SnF, R])

LookupError: 

===========================================================================
NLTK was unable to find the prover9 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on prover9, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

NLTK 的语义关系可以用标准的集合论方法表示：作为元组的集合。例如：假设我们有
一个域包括Bertie 、Olive 和Cyril，其中Bertie 是男孩，Olive 是女孩，而Cyril 是小狗。为了方便记述，我们用b、o 和c 作为模型中相应的标签。我们可以声明域如下：我们使用工具函数parse_valuation()将“符号=> 值”形式的字符串序列转换成一
个Valuation 对象

In [35]:
dom=set(['b','o','c'])

In [36]:
v = """
bertie => b
olive => o
cyril => c
boy => {b}
girl => {o}
dog => {c}
walk => {o, c}
see => {(b, o), (c, b), (o, c)}
"""

In [37]:
val = nltk.Valuation.fromstring(v)

In [38]:
print(val)

{'bertie': 'b',
 'boy': {('b',)},
 'cyril': 'c',
 'dog': {('c',)},
 'girl': {('o',)},
 'olive': 'o',
 'see': {('c', 'b'), ('o', 'c'), ('b', 'o')},
 'walk': {('c',), ('o',)}}


In [40]:
('o', 'c') in val['see']

True

使用构造函数Assignment赋值。
是一个从独立变量到域中实体的映射

In [41]:
('b',) in val['boy']

True

In [42]:
g = nltk.Assignment(dom,[('x','o'),('y','c')])

In [43]:
g

{'x': 'o', 'y': 'c'}

使用print()查看赋值

In [44]:
print (g)

g[c/y][o/x]


为一阶逻辑公式估值

In [45]:
m = nltk.Model(dom,val)#创建一个模型

In [46]:
m.evaluate('see(olive,y)',g)#调用evaluate（）方法计算真假

True

In [47]:
g['y']

'c'

In [48]:
m.evaluate('see(y,x)',g)

False

方法purge()从清除一个赋值中所有的绑定。

In [49]:
g.purge()

In [50]:
g

{}

In [51]:
m.evaluate('see(olive,y)',g)

'Undefined'

我们的模型已经包含了解释布尔运算的规则，任意复杂的公式都可以组合和评估

In [52]:
m.evaluate('see(bertie,olive) & boy(bertie) & -walk(bertie)',g)

True

In [53]:
 m.evaluate('exists x.(girl(x) & walk(x))', g)

True

In [54]:
m.evaluate('girl(x) & walk(x)', g.add('x', 'o'))

True

NLTK 中提供了一个有用的工具：satisfiers()方法。它返回满足开放公式的所有个体
的集合。该方法的参数是一个已分析的公式、一个变量和一个赋值。

In [55]:
fmla1 = read_expr('girl(x) | boy(x)')

In [56]:
 m.satisfiers(fmla1, 'x', g)

{'b', 'o'}

In [57]:
fmla2 = read_expr('girl(x) -> walk(x)')
m.satisfiers(fmla2, 'x', g)

{'b', 'c', 'o'}

In [58]:
fmla3 = read_expr('walk(x) -> girl(x)')
m.satisfiers(fmla3, 'x', g)

{'b', 'o'}

In [59]:
m.evaluate('all x.(girl(x) -> walk(x))', g)

True

为了检查这种歧义，我们对估值做如下修正

In [22]:
v2 = """
bruce => b
cyril => c
elspeth => e
julia => j
matthew => m
person => {b, e, j, m}
admire => {(j, b), (b, b), (m, e), (e, m), (c, a)}
"""

In [23]:
val2 = nltk.Valuation.fromstring(v2)

In [24]:
val2

{'admire': {('b', 'b'), ('c', 'a'), ('e', 'm'), ('j', 'b'), ('m', 'e')},
 'bruce': 'b',
 'cyril': 'c',
 'elspeth': 'e',
 'julia': 'j',
 'matthew': 'm',
 'person': {('b',), ('e',), ('j',), ('m',)}}

NLTK 中提供了一个有用的工具：satisfiers()方法。它返回满足开放公式
的所有个体的集合。该方法的参数是一个已分析的公式、一个变量和一个赋值

In [25]:
dom2 = val2.domain
m2 = nltk.Model(dom2, val2)
g2 = nltk.Assignment(dom2)
fmla4 = read_expr('(person(x) -> exists y.(person(y) & admire(x, y)))')#a中的开放式一阶逻辑
m2.satisfiers(fmla4, 'x', g2)#满足条件的所有x

{'a', 'b', 'c', 'e', 'j', 'm'}

In [26]:
dom2

{'a', 'b', 'c', 'e', 'j', 'm'}

In [27]:
fmla5 = read_expr('(person(y) & all x.(person(x) -> admire(x, y)))')#b中的一阶逻辑
m2.satisfiers(fmla5, 'y', g2)#满足条件的所有x

set()

In [28]:
fmla6 = read_expr('(person(y) & all x.((x = bruce | x = julia) -> admire(x, y)))')
m2.satisfiers(fmla6, 'y', g2)

{'b'}

In [60]:
a3 = read_expr('exists x.(man(x) & walks(x))')
c1 = read_expr('mortal(socrates)')
c2 = read_expr('-mortal(socrates)')
mb = nltk.Mace(5)
print(mb.build_model(None, [a3, c1]))

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [61]:
print(mb.build_model(None, [a3, c2]))

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [ ]:
print(mb.build_model(None, [c1, c2]))

In [62]:
a4 = read_expr('exists y. (woman(y) & all x. (man(x) -> love(x,y)))')
a5 = read_expr('man(adam)')
a6 = read_expr('woman(eve)')
g = read_expr('love(adam,eve)')
mc = nltk.MaceCommand(g, assumptions=[a4, a5, a6])
mc.build_model()

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [63]:
print(mc.valuation)

LookupError: You have to call build_model() first to get a model!

In [64]:
print(mc.valuation)

LookupError: You have to call build_model() first to get a model!

In [65]:
read_expr = nltk.sem.Expression.fromstring
expr = read_expr(r'\x.(walk(x) & chew_gum(x))')
expr

<LambdaExpression \x.(walk(x) & chew_gum(x))>

In [66]:
expr.free()

set()

In [67]:
print(read_expr(r'\x.(walk(x) & chew_gum(y))'))

\x.(walk(x) & chew_gum(y))


In [68]:
expr = read_expr(r'\x.(walk(x) & chew_gum(x))(gerald)')
print(expr)

\x.(walk(x) & chew_gum(x))(gerald)


In [69]:
print(expr.simplify())

(walk(gerald) & chew_gum(gerald))


In [70]:
 print(read_expr(r'\x.\y.(dog(x) & own(y, x))(cyril)').simplify())

\y.(dog(cyril) & own(y,cyril))


In [71]:
 print(read_expr(r'\x y.(dog(x) & own(y, x))(cyril, angus)').simplify())

(dog(cyril) & own(angus,cyril))


In [72]:
expr1 = read_expr('exists x.P(x)')
print(expr1)

exists x.P(x)


In [73]:
expr2 = expr1.alpha_convert(nltk.sem.Variable('z'))
print(expr2)

exists z.P(z)


In [74]:
expr1 == expr2

True

In [75]:
expr3 = read_expr('\P.(exists x.P(x))(\y.see(y, x))')

In [76]:
print(expr3)

(\P.exists x.P(x))(\y.see(y,x))


In [77]:
print(expr3.simplify())

exists z1.see(z1,x)


In [78]:
read_expr = nltk.sem.Expression.fromstring
tvp = read_expr(r'\X x.X(\y.chase(x,y))')
np = read_expr(r'(\P.exists x.(dog(x) & P(x)))')
vp = nltk.sem.ApplicationExpression(tvp, np)
print(vp)

(\X x.X(\y.chase(x,y)))(\P.exists x.(dog(x) & P(x)))


In [79]:
print(vp.simplify())

\x.exists z2.(dog(z2) & chase(x,z2))


In [80]:
from nltk import load_parser
parser = load_parser('grammars/book_grammars/simple-sem.fcfg', trace=0)
sentence = 'Angus gives a bone to every dog'
tokens = sentence.split()
for tree in parser.parse(tokens):
     print(tree.label()['SEM'])

all z4.(dog(z4) -> exists z3.(bone(z3) & give(angus,z3,z4)))


In [82]:
sents = ['Irene walks', 'Cyril bites an ankle']
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
for results in nltk.interpret_sents(sents, grammar_file):
    for (synrep, semrep) in results:
        print(synrep)

(S[SEM=<walk(irene)>]
  (NP[-LOC, NUM='sg', SEM=<\P.P(irene)>]
    (PropN[-LOC, NUM='sg', SEM=<\P.P(irene)>] Irene))
  (VP[NUM='sg', SEM=<\x.walk(x)>]
    (IV[NUM='sg', SEM=<\x.walk(x)>, TNS='pres'] walks)))
(S[SEM=<exists z5.(ankle(z5) & bite(cyril,z5))>]
  (NP[-LOC, NUM='sg', SEM=<\P.P(cyril)>]
    (PropN[-LOC, NUM='sg', SEM=<\P.P(cyril)>] Cyril))
  (VP[NUM='sg', SEM=<\x.exists z5.(ankle(z5) & bite(x,z5))>]
    (TV[NUM='sg', SEM=<\X x.X(\y.bite(x,y))>, TNS='pres'] bites)
    (NP[NUM='sg', SEM=<\Q.exists x.(ankle(x) & Q(x))>]
      (Det[NUM='sg', SEM=<\P Q.exists x.(P(x) & Q(x))>] an)
      (Nom[NUM='sg', SEM=<\x.ankle(x)>]
        (N[NUM='sg', SEM=<\x.ankle(x)>] ankle)))))


In [86]:
v = """
 bertie => b
 olive => o
 cyril => c
 boy => {b}
 girl => {o}
 dog => {c}
 walk => {o, c}
 see => {(b, o), (c, b), (o, c)}
 """
val = nltk.Valuation.fromstring(v)
g = nltk.Assignment(val.domain)
m = nltk.Model(val.domain, val)
sent = 'Cyril sees every boy'
grammar_file = 'grammars/book_grammars/simple-sem.fcfg'
results = nltk.evaluate_sents([sent], grammar_file, m, g)[0]
for (syntree, semrep, value) in results:
    print(semrep)
    print(value)

all z8.(boy(z8) -> see(cyril,z8))
True


In [87]:
from nltk.sem import cooper_storage as cs
sentence = 'every girl chases a dog'
trees = cs.parse_with_bindops(sentence, grammar='grammars/book_grammars/storage.fcfg')
semrep = trees[0].label()['SEM']
cs_semrep = cs.CooperStore(semrep)
print(cs_semrep.core)

chase(z2,z3)


In [88]:
for bo in cs_semrep.store:
     print(bo)

bo(\P.all x.(girl(x) -> P(x)),z2)
bo(\P.exists x.(dog(x) & P(x)),z3)


In [89]:
cs_semrep.s_retrieve(trace=True)

Permutation 1
   (\P.all x.(girl(x) -> P(x)))(\z2.chase(z2,z3))
   (\P.exists x.(dog(x) & P(x)))(\z3.all x.(girl(x) -> chase(x,z3)))
Permutation 2
   (\P.exists x.(dog(x) & P(x)))(\z3.chase(z2,z3))
   (\P.all x.(girl(x) -> P(x)))(\z2.exists x.(dog(x) & chase(z2,x)))


In [90]:
for reading in cs_semrep.readings:
    print(reading)

exists x.(dog(x) & all z11.(girl(z11) -> chase(z11,x)))
all x.(girl(x) -> exists z12.(dog(z12) & chase(x,z12)))


在NLTK 建立DRS 对象最简单的方法是通过解析一个字符串表示

In [91]:
read_dexpr = nltk.sem.DrtExpression.fromstring
drs1 = read_dexpr('([x, y], [angus(x), dog(y), own(x, y)])')
print(drs1)

([x,y],[angus(x), dog(y), own(x,y)])


In [92]:
drs1.draw()

每一个DRS 都可以转化为一阶逻辑公式，
fol()方法实现这种转换

In [93]:
print(drs1.fol())

exists x y.(angus(x) & dog(y) & own(x,y))


DRT 表达式有DRS连接运算符，用“+”符号表示
自动进行α-转换绑定变量避免名称冲突

In [94]:
drs2 = read_dexpr('([x], [walk(x)]) + ([y], [run(y)])')
print(drs2)

(([x],[walk(x)]) + ([y],[run(y)]))


In [95]:
print(drs2.simplify())

([x,y],[walk(x), run(y)])


In [96]:
drs3 = read_dexpr('([], [(([x], [dog(x)]) -> ([y],[ankle(y), bite(x, y)]))])')

In [97]:
print(drs3.fol())

all x.(dog(x) -> exists y.(ankle(y) & bite(x,y)))


In [98]:
drs4 = read_dexpr('([x, y], [angus(x), dog(y), own(x, y)])')
drs5 = read_dexpr('([u, z], [PRO(u), irene(z), bite(u, z)])')
drs6 = drs4 + drs5
print(drs6.simplify())

([u,x,y,z],[angus(x), dog(y), own(x,y), PRO(u), irene(z), bite(u,z)])


In [99]:
print(drs6.simplify().resolve_anaphora())

([u,x,y,z],[angus(x), dog(y), own(x,y), (u = [x,y,z]), irene(z), bite(u,z)])


In [100]:
from nltk import load_parser
parser = load_parser('grammars/book_grammars/drt.fcfg', logic_parser=nltk.sem.drt.DrtParser())
trees = list(parser.parse('Angus owns a dog'.split()))
print(trees[0].label()['SEM'].simplify())

([x,z15],[Angus(x), dog(z15), own(x,z15)])


段落是一个句子的序列s1, ... sn，段落线是读法的序列s1-ri, ... sn-rj，每个序列
对应段落中的一个句子。该模块按增量处理句子，当有歧义时保持追踪所有可能的线。

In [101]:
dt = nltk.DiscourseTester(['A student dances', 'Every student is a person'])

In [102]:
dt.readings()

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [103]:
dt.add_sentence('No person dances', consistchk=True)

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [104]:
dt.retract_sentence('No person dances', verbose=True)

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [105]:
dt.add_sentence('A person dances', informchk=True)

LookupError: 

===========================================================================
NLTK was unable to find the mace4 file!
Use software specific configuration paramaters or set the PROVER9 environment variable.

  Searched in:
    - /usr/local/bin/prover9
    - /usr/local/bin/prover9/bin
    - /usr/local/bin
    - /usr/bin
    - /usr/local/prover9
    - /usr/local/share/prover9

  For more information on mace4, see:
    <http://www.cs.unm.edu/~mccune/prover9/>
===========================================================================

In [109]:
from nltk.tag import RegexpTagger
tagger = RegexpTagger(
     [('^(chases|runs)$', 'VB'),
      ('^(a)$', 'ex_quant'),
      ('^(every)$', 'univ_quant'),
      ('^(dog|boy)$', 'NN'),
      ('^(He)$', 'PRP')
 ])
rc = nltk.DrtGlueReadingCommand(depparser=nltk.MaltParser(tagger=tagger))
dt = nltk.DiscourseTester(['Every dog chases a boy', 'He runs'], rc)
dt.readings()

TypeError: __init__() missing 1 required positional argument: 'parser_dirname'

In [ ]:
dt.readings(show_thread_readings=True)

In [ ]:
dt.readings(show_thread_readings=True, filter=True)